<a href="https://colab.research.google.com/github/natadekoko68/signate/blob/984/%E8%87%AA%E5%8B%95%E8%BB%8A%E3%81%AE%E8%B5%B0%E8%A1%8C%E8%B7%9D%E9%9B%A2%E4%BA%88%E6%B8%AC(%E6%8F%90%E5%87%BA19_23).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("content/")

Mounted at content/


In [75]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
import numpy as np
import warnings
warnings.simplefilter('ignore')

In [62]:
train = pd.read_csv("/content/content/MyDrive/Signate_data/2308自動車の走行距離予測/train.csv")
test = pd.read_csv("/content/content/MyDrive/Signate_data/2308自動車の走行距離予測/test.csv")
sample_submit = pd.read_csv("/content/content/MyDrive/Signate_data/2308自動車の走行距離予測/sample_submit.csv",header=None)
train_len = len(train.reset_index(drop=True))
df = pd.concat([train,test],axis=0)
df["car name"] = df["car name"].str.split().apply(lambda x: x[0])
df.loc[df["car name"]=='vokswagen', "car name"] = 'volkswagen'
df.loc[df["car name"]=='vw', "car name"] = 'volkswagen'
df.loc[df["car name"]=='toyouta', "car name"] = 'toyota'

car_manufacturer_country = {
    'amc': 'USA',
    'audi': 'Germany',
    'buick': 'USA',
    'capri': 'USA',
    'chevrolet': 'USA',
    'chevy': 'USA',
    'datsun': 'Japan',
    'dodge': 'USA',
    'fiat': 'Italy',
    'ford': 'USA',
    'hi': 'USA',
    'honda': 'Japan',
    'mercury': 'USA',
    'oldsmobile': 'USA',
    'opel': 'Germany',
    'peugeot': 'France',
    'plymouth': 'USA',
    'pontiac': 'USA',
    'renault': 'France',
    'saab': 'Sweden',
    'toyota': 'Japan',
    'triumph': 'UK',
    'volkswagen': 'Germany',
    'volvo': 'Sweden'
}
df["country"] = df["car name"].map(car_manufacturer_country)

df.loc[df["horsepower"]=="?","horsepower"]  = df.loc[df["horsepower"]!="?","horsepower"].astype(int).mean()
df = df.drop(["id"],axis=1)

encoded_df = pd.get_dummies(df, columns=['country',"car name"], prefix=['country',"car name"]).reset_index(drop=True)
temp = encoded_df.loc[:train_len-1,]
X = temp.drop(["mpg"],axis=1)
y = temp["mpg"]
test = encoded_df.loc[train_len:,:]
test = test.drop(["mpg"],axis=1)


In [81]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
forest = RandomForestRegressor(n_estimators=10000,
                            #    criterion='',
                               max_depth=None,
                               min_samples_split=2,
                               min_samples_leaf=1,
                               min_weight_fraction_leaf=0.0,
                               max_leaf_nodes=None,
                               min_impurity_decrease=0.0,
                               max_features="sqrt",
                               bootstrap=True,
                               oob_score=False,
                               n_jobs=-1,
                               random_state=42,
                               verbose=0,
                               warm_start=False,
                               ccp_alpha=0.0,
                               max_samples=None
                              )
forest.fit(X_train, y_train)
y_train_pred = forest.predict(X_train)
y_test_pred  = forest.predict(X_valid)

print('RMSE 学習: %.2f, テスト: %.2f' % (
        mean_squared_error(y_train, y_train_pred, squared=False), # 学習
        mean_squared_error(y_valid, y_test_pred, squared=False)    # テスト
      ))

RMSE 学習: 1.03, テスト: 3.29


In [82]:
forest = RandomForestRegressor(n_estimators=10000,
                            #    criterion='',
                               max_depth=None,
                               min_samples_split=2,
                               min_samples_leaf=1,
                               min_weight_fraction_leaf=0.0,
                               max_leaf_nodes=None,
                               min_impurity_decrease=0.0,
                               max_features="sqrt",
                               bootstrap=True,
                               oob_score=False,
                               n_jobs=-1,
                               random_state=42,
                               verbose=0,
                               warm_start=False,
                               ccp_alpha=0.0,
                               max_samples=None
                              )
forest.fit(X, y)

y_test_pred = forest.predict(test)


,1,0.0
0,2,0.0
1,5,0.0
2,6,0.0
3,8,0.0
4,10,0.0
...,...,...
494,992,0.0
495,993,0.0
496,996,0.0
497,998,0.0


In [83]:
sample_submit = pd.read_csv("/content/content/MyDrive/Signate_data/2308自動車の走行距離予測/sample_submit.csv",header=None)
sub = pd.Series(y_test_pred)
sample_submit[1] = sub
sample_submit.to_csv("submit2.csv",header=None,index=False)

In [74]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print("Best Hyperparameters:", best_params)
print("Best RMSE:", np.sqrt(-grid_search.best_score_))


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors

Best Hyperparameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best RMSE: 3.0244370488325196
